In [1]:
import astropy.units as u

from altaipony.lcio import from_mast
from altaipony.altai import find_iterative_median
from altaipony.flarelc import FlareLightCurve


from lightkurve import search_targetpixelfile

import matplotlib.pyplot as plt
%matplotlib inline

import copy


from scipy.interpolate import UnivariateSpline

import os
import pandas as pd
import numpy as np

from funcs.funcs import read_custom_aperture_lc
from funcs.custom_detrending import custom_detrending

from scipy import optimize

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
clcs = ! ls /home/ekaterina/Documents/001_Science/TESS_UCDs/TESS_UCD_flares/custom_aperture/*fits
clcs = [x.split("/")[-1] for x in clcs]

def get_customlc(TIC, c, clcs):
    for p in clcs:
        if (str(TIC) in p.split("-")[2]) & (c == int(p.split("-")[1][1:])):
            print(p)
            pac = p
    return pac


In [ ]:
CWD = "/".join(os.getcwd().split("/")[:-1])
respath = f"{CWD}/flare_tables/2019_11_21_TESSUCDs_flares.csv"
df = pd.read_csv(respath)
targetswflare = df.TIC.unique().shape[0]
LCswflares = df.groupby(by=["TIC","Campaign"]).count().sort_values(by="ampl_rec").shape[0]
candidates = df.shape[0]
amplmax = df.ampl_rec.max()


In [ ]:
plt.scatter(df.tstop-df.tstart, df.ampl_rec)
plt.xscale("log")
plt.yscale("log")

In [ ]:
plt.scatter(df.ampl_rec, df.ed_rec)
plt.xscale("log")
plt.yscale("log")

In [ ]:
df[df.ed_rec<1]

In [ ]:
df[df.Campaign==6].groupby("TIC").count()
TIC = 389051009
#df.groupby("TIC").Campaign.count().sort_values(ascending=False)[:10]
df[df.TIC==TIC].Campaign.unique()
#df.sort_values(by="ed_rec", ascending=False).head()

In [ ]:
#c=1

#path = get_customlc(TIC, c, clcs)

#path = f"{CWD}/custom_aperture/" + path

#flc = read_custom_aperture_lc(path)

flck = copy.deepcopy(flc)
flcd = custom_detrending(flc)
flcd = flcd.find_flares()

#flcd.flares.sort_values(by="ed_rec",ascending=False)

fig, ax = plt.subplots(ncols=1,nrows=1, figsize=(16,5))
g = 14

ax.plot(flck.time, flck.flux, c="orange", label=f"TIC {flc.targetid}, S{flc.campaign}");#flc.time, 
ax.plot(flcd.time,flcd.it_med+3*flcd.detrended_flux_err, c="r");
ax.plot(flcd.time,flcd.detrended_flux, c="g");
for i, f in flcd.flares.iterrows():
    cond = ((flcd.cadenceno >= f.cstart) & (flcd.cadenceno <= f.cstop))
    end = np.where(flcd.cadenceno == f.cstop)[0][0]
    sta = np.where(flcd.cadenceno == f.cstart)[0][0]
    print(end, sta, flcd.time[sta],flcd.time[end],)
    ax.axvspan(flcd.time[sta]-.5,flcd.time[end]+.5,alpha=.4

            )
#f = flcd.flares.iloc[2]
#plt.xlim(f.tstart-1.1,f.tstop+1.1);
#plt.xlim(f.cstart-1000,f.cstop+1000);
#plt.ylim(1600,2000)
#plt.xlim(298200,298600)
#plt.xlim(1641.81,1644)
plt.xlabel("time",fontsize=14)
plt.ylabel("flux",fontsize=14)
plt.legend(fontsize=14)
plt.savefig(f"{CWD}/plots/TIC{flc.targetid}_s{flc.campaign}.png",dpi=300)
plt.close()

In [ ]:
from altaipony.fakeflares import aflare
#for i,f in flcd.flares.iterrows():
plt.figure(figsize=(16,5))
g = 14

#plt.plot(flc.time, flc.flux, c="orange");#flc.time, 
plt.plot(flck.time, flck.flux, c="grey")
#plt.plot(flcd.time,flcd.it_med+3*flcd.detrended_flux_err, c="r");
#plt.plot(flcd.time,flcd.detrended_flux, c="g");
f = flcd.flares.iloc[1]
plt.xlim(f.tstart-1.1,f.tstop+1.1);
#plt.xlim(1386.142,1386.15)
#for a in np.linspace(.9,1.7,10):
#    for d in np.linspace(.4,.6,10):
#        plt.plot(flcd.time,flcd.it_med*aflare(flcd.time, 1641.835, d/6, a,) + flcd.it_med, c="grey", alpha=.05)
#plt.title("ampl=1.9-2.7, dur=.4-.6 d")
#plt.savefig("/home/ekaterina/Documents/001_Science/TESS_UCDs/TESS_UCD_flares/weird_feature/flarefit1.png", dpi=300)
i = 12
#plt.xlim(flck.time[0]+i*2,flck.time[0]+2+i*2)
plt.ylim(600,800)
#plt.xlim(1331.5,1333)

In [ ]:
def cosine(x, a, b, c, d):
    return a * np.cos(b * x + c) + d 
    
#flck = copy.deepcopy(flc)
cond = np.invert(np.isnan(flck.time)) & np.invert(np.isnan(flck.flux)) 
p, p_cov = optimize.curve_fit(cosine, flck.time[cond], flck.flux[cond],
                                          p0=[np.nanstd(flck.flux),
                                          2*np.pi*pg.frequency_at_max_power.value,
                                          0, np.nanmean(flck.flux)])
#flck.flux = np.nanmean(flck.flux) + flck.flux-cosine(flck.time, p[0], p[1], p[2], p[3])
rel_amplitude = p[0] / np.nanmedian(flck.flux)
print(f"Relative amplitude of modulation: {rel_amplitude:.1e}")
plt.figure(figsize=(15,6))
plt.plot(flck.time, flck.flux,c="grey")
#plt.plot(flck.time, np.nanmean(flck.flux) + flck.flux - cosine(flck.time, p[0], p[1], p[2], p[3]))
plt.plot(flck.time, cosine(flck.time, 1*p[0], p[1], p[2], p[3]), c="navy") #+ 4*(1340.6- flck.time),
plt.ylim(610,800)
plt.xlim(1339.7,1341.7)
plt.xlim(1331.5,1333.5)
#plt.xlim(1340.225,1340.34)
plt.title("TIC 389051009, sector 1")
plt.savefig(f"{CWD}/multiperiod_flares/{TIC}_{c}/plots/{TIC}_s{c}_sinusoidal2.png",dpi=300)

In [ ]:
(1340.225-1340.34)/0.14316634

In [ ]:
per = 0.41302504
minim = 1625.28

flcd.flares["phases"] = ((flcd.flares.tstart - minim) % per) / per
flcd.flares["phasef"] = ((flcd.flares.tstop - minim) % per) / per
#plt.hist(flcd.flares.phases, bins=5)
plt.xlim(0,1)
#flcd.flares
for i,row in flcd.flares.iterrows():
    plt.plot([row.phases,row.phasef],[1+i]*2)

In [ ]:
flcd.flares

In [ ]:
c=6

#path = get_customlc(TIC, c, clcs)
for path in clcs[3:]:
    path = f"{CWD}/custom_aperture/" + path

    flc = read_custom_aperture_lc(path)
    flck = copy.deepcopy(flc)
    flcd = custom_detrending(flc)
    flcd = flcd.find_flares()

    #flcd.flares.sort_values(by="ed_rec",ascending=False)

    fig, ax = plt.subplots(ncols=1,nrows=1, figsize=(16,5))
    g = 14

    ax.plot(flck.time, flck.flux, c="orange", label=f"TIC {flc.targetid}, S{flc.campaign}");#flc.time, 
    ax.plot(flcd.time,flcd.it_med+13*flcd.detrended_flux_err, c="r");
    ax.plot(flcd.time,flcd.detrended_flux+10*flcd.detrended_flux_err, c="g");
    for i, f in flcd.flares.iterrows():
        cond = ((flcd.cadenceno >= f.cstart) & (flcd.cadenceno <= f.cstop))
        end = np.where(flcd.cadenceno == f.cstop)[0][0]
        sta = np.where(flcd.cadenceno == f.cstart)[0][0]
      #  print(end, sta, flcd.time[sta],flcd.time[end],)
        ax.axvspan(flcd.time[sta]-.5,flcd.time[end]+.5,alpha=.4
                
                )
    #f = flcd.flares.iloc[2]
    #plt.xlim(f.tstart-1.1,f.tstop+1.1);
    #plt.xlim(f.cstart-1000,f.cstop+1000);
    #plt.ylim(1600,2000)
    #plt.xlim(298200,298600)
    #plt.xlim(1641.81,1644)
    plt.xlabel("time",fontsize=14)
    plt.ylabel("flux",fontsize=14)
    plt.legend(fontsize=14)
    plt.savefig(f"{CWD}/plots/TIC{flc.targetid}_s{flc.campaign}.png",dpi=300)
    plt.close()

In [ ]:
CWD = "/".join(os.getcwd().split("/")[:-1])
respath = f"{CWD}/flare_tables/2019_11_18_TESSUCDs_flares.csv"
df = pd.read_csv(respath)
f = df
f["x"] =f.tstop-f.tstart
f = f.sort_values(by="x")
x = (f.tstop-f.tstart).values
x = f.ed_rec.values
y = f.ampl_rec.values
plt.scatter(x, y)
plt.scatter(flcd.flares.ed_rec, flcd.flares.ampl_rec)
#plt.scatter(flcd.flares.tstop-flcd.flares.tstart, flcd.flares.ed_rec)
plt.xscale("log")
plt.yscale("log")
#np.polyfit(f.tstop-f.tstart, f.ed_rec,1)
from scipy.optimize import curve_fit
def func(x, a, b):
    return a * np.power(x , b)
popt, pcov = curve_fit(func, x, y)
x=np.array([5e-4,5e-3,5e-2,5e-1])
plt.plot(x, func(x, *popt), 'r-',
         label='fit: a=%5.3f, b=%5.3f' % tuple(popt))
plt.legend()
func(0.4, *popt),func(0.6, *popt)

In [ ]:
f = flcd.flares.loc[[0,1,5],:]
f["x"] =f.tstop-f.tstart
f = f.sort_values(by="x")
x = (f.tstop-f.tstart).values
y = f.ampl_rec.values
plt.scatter(x, y)
plt.xscale("log")
plt.yscale("log")
#np.polyfit(f.tstop-f.tstart, f.ed_rec,1)
from scipy.optimize import curve_fit
def func(x, a):
    return a * np.power(x , .965)#b=1.465 for edrec, b=0.965 for ampl_rec
popt, pcov = curve_fit(func, x, y)
x=np.array([5e-4,5e-3,5e-2,5e-1])
plt.plot(x, func(x, *popt), 'r-',
         label='fit: a=%5.3f' % tuple(popt))
plt.legend()
func(0.4, *popt),func(0.6, *popt)
#ED = (7981.423328873443, 14456.196424545316) s, amplitude=(1.856542468781827, 2.7455727338847487)

In [ ]:
tpf = search_targetpixelfile("TIC" + str(flcd.targetid), sector=c).download()

In [4]:
tpf = search_targetpixelfile("TIC 237880881", sector=1).download()

10% (1972/20076) of the cadences will be ignored due to the quality mask (quality_bitmask=175).
10% (1972/20076) of the cadences will be ignored due to the quality mask (quality_bitmask=175).


In [5]:
tpf.interact()

INFO:bokeh.server.server:Starting Bokeh server version 1.4.0 (running on Tornado 6.0.3)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


200 GET /autoload.js?bokeh-autoload-element=1196&bokeh-absolute-url=http://localhost:38437&resources=none (127.0.0.1) 950.89ms
101 GET /ws?bokeh-protocol-version=1.0&bokeh-session-id=ABy1JBV2aqZ9IA7q9LtKeBK6v1Epm5hJMjwBOadmBTyV (127.0.0.1) 0.52ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created


In [ ]:
tpf.plot(cadenceno=251071)

358108509 1 last flare suffers from de-trending being weird at the end of the LC
340703996 2 first flare could also be an undetrended wiggle in the LC

In [ ]:
pg = flck.remove_nans().to_periodogram(freq_unit=1/u.d,
                                          maximum_frequency=10,
                                          minimum_frequency=0.5)
pg.plot()
plt.xlim(.5, 10)
period = (1 / pg.frequency_at_max_power).to("h")
period.to("d")